In [81]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [53]:
caliper = pd.read_csv(
    '../data/test/caliper_output.txt', 
    sep='|',
    header=0,
    dtype={
        'displacement': np.float64,
        'delay': np.float64
    },
    parse_dates=['time']
)

scale = pd.read_csv(
    "../data/test/scale_output.txt",
    sep='|',
    header=0,
    dtype={
        'weight': np.float64,
        'unit': str,
        'flag': str
    },
    parse_dates=['time']
)

In [ ]:
def read_data(caliper_path, scale_path):
    """ Read caliper and scale data """
    caliper = pd.read_csv(
        caliper_path, 
        sep='|',
        header=0,
        dtype={
            'displacement': np.float64,
            'delay': np.float64
        },
        parse_dates=['time']
    )

    scale = pd.read_csv(
        scale_path,
        sep='|',
        header=0,
        dtype={
            'weight': np.float64,
            'unit': str,
            'flag': str
        },
        parse_dates=['time']
    )
    return caliper, scale

In [54]:
def clean_raw_data(caliper:pd.DataFrame, scale:pd.DataFrame):
    """ Clean raw data heads and tails for data matching later """
    scale_start_ind = 0
    while (caliper.time < scale.time.iloc[scale_start_ind]).sum() == 0:
        scale_start_ind += 1
    caliper_start_ind = caliper.index[caliper.time < scale.time.iloc[scale_start_ind]].max()

    scale_end_ind = scale.index.max()
    while (caliper.time > scale.time.iloc[scale_end_ind]).sum() == 0:
        scale_end_ind -= 1
    caliper_end_index = caliper.index[caliper.time > scale.time.iloc[scale_end_ind]].min()
    clean_scale = scale.iloc[scale_start_ind:scale_end_ind + 1]
    clean_caliper = caliper.iloc[caliper_start_ind:caliper_end_index + 1]

    return clean_caliper.reset_index(drop=True), clean_scale.reset_index(drop=True)

In [55]:
ccaliper, cscale = clean_raw_data(caliper=caliper, scale=scale)

In [77]:
def interpolate_displacement(caliper:pd.DataFrame, scale:pd.DataFrame):
    """ 
    Interpolate displacement data to match weight readings.
    Assuming slow change in displacement, the interpolation should be decently accurate.
    """
    clpr = np.zeros(scale.shape[0])
    row_iter = scale.iterrows()
    for row in row_iter:
        # get index for interpolate
        target_time = row[1].time
        clwr_ind = caliper.index[caliper.time < target_time].max()
        cupr_ind = clwr_ind + 1
        # interpolate
        dt_range = (caliper.time.iloc[cupr_ind] - caliper.time.iloc[clwr_ind]).total_seconds()
        dt = (target_time - caliper.time.iloc[clwr_ind]).total_seconds()
        dx_range = caliper.displacement.iloc[cupr_ind] - caliper.displacement.iloc[clwr_ind]
        clpr[row[0]] = dt / dt_range * dx_range + caliper.displacement.iloc[clwr_ind]
    return clpr, scale.weight.to_numpy()

In [78]:
xls, wls = interpolate_displacement(caliper=ccaliper, scale=cscale)

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.86481992e-03, 3.86662140e+00, 4.30375594e+00, 4.32243265e+00,
       5.34867129e+00, 6.38913781e+00, 7.55436678e+00, 8.97269003e+00,
       1.04374626e+01, 1.07623543e+01, 1.08533322e+01, 1.00878396e+01,
       1.01196462e+01, 1.13171793e+01, 1.27610172e+01, 1.35452972e+01,
       1.40472360e+01, 1.44183763e+01, 1.49704153e+01, 1.49720000e+01,
       1.49691484e+01, 1.49680000e+01, 1.49700000e+01, 1.49700000e+01,
       1.56701532e+01])

In [69]:
r[1].time

Timestamp('2024-08-15 16:27:14.610912')